In [1]:
cd ..


/home/niel


In [2]:
import sys
!{sys.executable} -m pip install requests --q
!{sys.executable} -m pip install bs4 --q
!{sys.executable} -m pip install numpy --q

In [3]:
import numpy as np
import pandas as pd
import json  
import pickle
import requests
import lxml
from bs4 import BeautifulSoup
import os
import zipfile


In [4]:
ls


aoe_data/     Dokumentenanalyse/  fastai_nbdev_test/  Untitled.ipynb
aoe_data.zip  drive/              nbdev/              zara/
deepflash2/   drive_files/        Untitled1.ipynb


**Data and Settings**


In [5]:
#settings
scrape_player_names = True
scrape_matches= True
parallel_download = True


#directorys
player_file_path = "aoe_data/data/scraped_matches/players.pkl"
player_elo_path = "aoe_data/masterdata/player_elo.pkl"

found_matches_filepath = "aoe_data/data/scraped_matches/found_recordfile_matches.pkl"

zip_path = "aoe_data/data/scraped_matches/zipped_recordfiles/"
unzip_path = "aoe_data/data/scraped_matches/unzipped_recordfiles/"

In [6]:
#find already scraped matches for parallel download
scraped_matches = []

for file in os.listdir(zip_path):
    filename = os.fsdecode(file)

    match_id = filename.partition("=")[2].partition("&")[0]
    scraped_matches.append(match_id)

**Functions**


In [7]:
def scrape_gameinfo_by_playername(p1_name: str,p2_name: str):
    #search for p1 name on aoe2insights
    source = requests.get('https://www.aoe2insights.com/search/?q={}'.format(p1_name)).text
    soup = BeautifulSoup(source, 'html.parser')

    user_string = soup.find('a', class_='stretched-link')#.__dict__.keys()
    p1_id = user_string['href'].split('/')[2]

    #search for RM 1v1 matches with p2
    source = requests.get('https://www.aoe2insights.com/user/{}/matches/?ladder=3&player={}'.format(p1_id,p2_name)).text
    soup = BeautifulSoup(source, 'html.parser')

    match_string = soup.find('a', class_='float-right stretched-link')#.__dict__.keys()
    match_id = match_string['href'].split('/')[2]
    
    return {'p1_id': p1_id, 'match_id': match_id}

def scrape_gameinfo_by_playerid(p1_id: int,p2_id: int):
    #search for RM 1v1 matches with p2
    source = requests.get('https://www.aoe2insights.com/user/{}/matches/?ladder=3&player={}'.format(p1_id,p2_name)).text
    soup = BeautifulSoup(source, 'html.parser')

    match_string = soup.find('a', class_='float-right stretched-link')#.__dict__.keys()
    match_id = match_string['href'].split('/')[2]
    
    return {'p1_id': p1_id, 'match_id': match_id}


def download_file(url, save_path):
    local_filename = save_path + url.split('/')[-1]
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)

    return local_filename


def download_match(match_id, pA_id, pB_id, zip_path):
    #check if two users were passed:
    if pA_id == pB_id:
        two_user = False
    else: two_user = True
    
    #create url
    url = "https://aoe.ms/replay/?gameId={}&profileId={}".format(str(match_id),str(pA_id))
    response = requests.get(url).status_code

    if response == 404:
        if two_user:
            new_url = "https://aoe.ms/replay/?gameId={}&profileId={}".format(str(match_id),str(pB_id))
            response = requests.get(url).status_code

            if response == 404:
                found = False
            else:
                try:
                    download_file(new_url, zip_path)
                    found = True
                except:
                    found = False
        else: found = False
    else:
        try:
            download_file(url, zip_path)
            found = True
        except:
            found = False
        
    return found, response


def print_txt(msg: str):
    print(msg, end = '\r')
    text_file = open("aoe_data/Outputs/Download_Output.txt", "w")
    text_file.write(msg)
    text_file.close()

**Scrape**

In [8]:
player_elos = {}


#scrape player names
if scrape_player_names:
    players = {}
    errors = 0
    page_number = 1
    max_page_number = 1723

    while page_number < max_page_number:
        try:
            #scrape aoe2insights
            source = requests.get('https://www.aoe2insights.com/leaderboard/3/?page={}'.format(str(page_number))).text
            soup = BeautifulSoup(source, 'html.parser')
            table = soup.find('table','table table-striped table-dark table-responsive-sm')

            for row in table.findAll("tr"):
                #get user id
                cells = row.findAll("a")
                for e in cells:
                    if "/user/" in str(e['href']):
                        user_id = e['href'].replace('/user/','').replace('/','')
                        username = e.string.strip()
                        players[user_id] = username
                
                #get user rating
                cells = row.findAll("td")
                for e in cells: 
                    elem = e.next_element.strip()
                    if elem.isnumeric():
                        player_elos[int(user_id)] = int(elem)
            #persistate
            if page_number % 50 == 0:
                output = open(player_file_path , 'wb')
                pickle.dump(players, output)
                output.close()
        except:
            errors +=1

        print("Parsed:" , page_number, " Pages - error on: ", errors,"- Found Players:",len(players),  end='\r')
        page_number +=1

    #final persistation
    output = open(player_file_path , 'wb')
    pickle.dump(players, output)
    output.close()

    #final persistation
    output = open(player_elo_path , 'wb')
    pickle.dump(player_elos, output)
    output.close()
        
else:
    with open(player_file_path, 'rb') as f:
        players = pickle.load(f)
    
print(len(players), " Players found.")

39906  Players found. error on:  1322 - Found Players: 3990639906- Found Players: 399061493  Pages - error on:  1093 - Found Players: 39906


In [9]:
   
start_msg = "Start Download"
print_txt(start_msg)


In [ ]:
matches = {}
errors = 0
max_pages = 1
directory = "aoe_data/data/scraped_matches/"
file_path_matches = directory + "matches.pkl"
file_path_scraped_players = directory +"already_scraped_players.pkl"
scraped_players = []
scraped_files = 0
failed_files = 0
response = ""

if scrape_matches:
    for index, user_id in enumerate(players):
        try:
            #reset parameter
            page_number = 1
            
            #only scrape first n pages of games
            while page_number<=max_pages:
                #make request
                response = requests.get("https://www.aoe2insights.com/user/{}/matches/?ladder=3&page={}".format(str(user_id),str(page_number)))
                source = response.text
                soup = BeautifulSoup(source, 'html.parser')

                #check if page has matches on it
                warning = soup.find('div',class_ ='col-12 alert alert-warning')
                if warning is not None or response.status_code == 404:
                    break

                #get match_id
                match_links = soup.findAll('a',class_='float-right stretched-link')
                for e in match_links:
                    match_id = e['href'].replace('/match/','').replace('/','')

                    #fill data
                    if match_id in matches:
                        matches[match_id]['pB_id'] = user_id
                        
                        if parallel_download:
                            #try to scrape recordfiles when full info is gathered
                            if match_id not in scraped_matches:
                                try:
                                    found, response = download_match(match_id, matches[match_id]['pA_id'], matches[match_id]['pB_id'], zip_path)
                                    if found:
                                        scraped_files +=1
                                        #reduce fails bc it was counted on first attempt
                                        failed_files -= 1
                                    else:
                                        failed_files += 1
                                except:
                                    failed_files +=1
                    else:
                        matches[match_id] = {}
                        matches[match_id]['match_id'] = match_id 
                        matches[match_id]['pA_id'] = user_id
                        matches[match_id]['pB_id'] = None
                        
                        if parallel_download:
                            if match_id not in scraped_matches:
                                    try:
                                        found, response = download_match(match_id, matches[match_id]['pA_id'], matches[match_id]['pA_id'], zip_path)
                                        if found:
                                            scraped_files +=1
                                        else:
                                            failed_files += 1
                                    except:
                                        failed_files +=1
                    
                    if user_id not in scraped_players:
                        scraped_players.append(user_id)
                    
                    #persistate
                    if len(matches) % 1000 == 0:
                        output = open(directory + "over_time/" + str(len(matches))+ "_matches.pkl" , 'wb')
                        pickle.dump(matches, output)
                        output.close()
                        
                        output = open(directory + "over_time/" + str(len(matches))+ "_already_scraped_players.pkl" , 'wb')
                        pickle.dump(scraped_players, output)
                        output.close()
                        
                        print_txt("Parsed:" + str(index) + " Players - error on: " + str(errors) +"- Found Matches:"+ str(len(matches)) + " - Found Files: " + str(scraped_files)+  " - Failed Files: " +str(failed_files))
   
                print("Parsed:" , index, " Players - error on: ", errors,"- Found Matches:",len(matches), " - Found Files: ",str(scraped_files), " - Failed Files: ",str(failed_files), " - Last Response: ",response,end='\r')
                page_number +=1
                
        except:
            errors +=1
            print("Parsed:" , index, " Players - error on: ", errors,"- Found Matches:",len(matches), " - Found Files: ",str(scraped_files), " - Failed Files: ",str(failed_files),  end='\r')
            pass
    
    #final persistation
    output = open(file_path_matches , 'wb')
    pickle.dump(matches, output)
    output.close()
    
else:
    with open(file_path_matches, 'rb') as f:
        matches = pickle.load(f)

print(len(matches), " Matches Found")

In [ ]:
matches_df = pd.DataFrame.from_dict(matches, orient='index')
print(matches_df.shape)
matches_df.head()